In [1]:
import pyabc
import scipy
import numpy as np
from pyabc.visualization import plot_kde_matrix_highlevel,plot_kde_matrix2,plot_kde_matrix_for_paper
from datetime import datetime
import pandas as pd
from arviz import hdi
from pyabc.visualization import plot_kde_matrix_highlevel, plot_kde_matrix
import numpy as np
import matplotlib.pyplot as plt
path = '/Users/adsurbum/sim-data/single-model-abc/new4'

f_no_aneuploidy = '2021-12-29-no-aneuploidy'
f_with_aneuploidy = '2021-12-29-an200-k1'
f_tau100 = '2022-01-12-tau100-r1k'
f_tau10 = '2021-12-20-an200-k10'

history = pyabc.History("sqlite:///{0}/{1}/{1}.db".format(path,f_no_aneuploidy))
history.get_all_populations()[-3:]

,t,population_end_time,samples,epsilon,particles
31,30,2022-01-06 20:40:28.751281,9066,0.275595,14
32,31,2022-01-08 21:03:27.959201,26766,0.272928,11
33,32,2022-01-12 13:03:32.838836,53034,0.267157,14


In [2]:
history = pyabc.History("sqlite:///{0}/{1}/{1}.db".format(path,f_with_aneuploidy))
history.get_all_populations()[-3:]

,t,population_end_time,samples,epsilon,particles
4,3,2021-12-29 16:37:20.673694,16125,0.145762,1111
5,4,2021-12-29 21:50:47.240188,27629,0.131083,964
6,5,2021-12-30 08:00:52.829615,53803,0.126588,982


In [3]:
def roundd(v):
    return round(v,3)
        
def findd(n):
    for i in range(0,12):
        m = n*10**i
        if m>=1 and m<=10:
            return i
    raise ValueError(n)
           
# returns (MAP, low_hdi, high_hdi)
def calc_hdi(f, hdi_p=.50):
    history = pyabc.History("sqlite:///{0}/{1}/{1}.db".format(path, f))
    samples, weights = history.get_distribution(m=0, t=None)
    columns = list(samples.columns)
    kde = scipy.stats.gaussian_kde(samples.T.values.tolist(), weights=weights)
    size = 50000
    samples = kde.resample(size).T
    samples = pd.DataFrame(data=samples)
    MAP = scipy.optimize.minimize(lambda x: -kde.logpdf(x) ,samples.median().values)['x']
    hdi_res = hdi(samples.values, hdi_prob=hdi_p)
    hi = hdi_res.T[1]
    lo = hdi_res.T[0]
    return (MAP, lo, hi)

def print_mode_and_hdi(f, display_cols=['\mu','\delta','w_{2n+1}','w_{2n+1^*}', 'w_{2n^*}'], hdi_p=.50):
    ans = calc_hdi(f,hdi_p)
    orders = [np.array(list(map(findd,ans))) for ans in ans]
    singles = [list(map(roundd,ans*(10**np.array(list(map(findd,ans)))))) for ans in ans]
    for a in list(zip(list(display_cols),*singles, *orders)):
        if a[-1]==0:
            print('${}={} ({}-{})$,'.format(*a))
        else:
            print('${0}={1}\\times10^{{-{4}}} ({2}\\times10^{{-{5}}}-{3}\\times10^{{-{6}}})$,'.format(*a))
    

import warnings
warnings.filterwarnings('ignore')

In [26]:
print_mode_and_hdi(f_no_aneuploidy, display_cols = ['\mu', 'w_{2n^*}'])

$\mu=7.982\times10^{-9} (7.919\times10^{-9}-8.149\times10^{-9})$,
$w_{2n^*}=1.013 (1.012-1.013)$,


In [27]:
print_mode_and_hdi(f_with_aneuploidy, display_cols = ['\mu','\delta','w_{2n+1}','w_{2n+1^*}', 'w_{2n^*}'])

$\mu=2.945\times10^{-6} (2.232\times10^{-7}-4.152\times10^{-6})$,
$\delta=1.723\times10^{-3} (1.347\times10^{-3}-2.709\times10^{-3})$,
$w_{2n+1}=1.022 (1.021-1.023)$,
$w_{2n+1^*}=1.025 (1.023-1.026)$,
$w_{2n^*}=1.028 (1.026-1.029)$,


In [31]:
w1,w2,w3 = calc_hdi(f_with_aneuploidy)[0][2:]
w1,w2,w3 = 1.022, 1.025, 1.028
s = w3 -1
c = 1-(w2-w1)/(w3-1) 
b = w3-1+c
round(s,3),round(b,3),round(c,3)

(0.028, 0.921, 0.893)

In [28]:
calc_hdi(f_tau10,hdi_p=.5)

(array([1.66101248e-06, 1.47758352e-03, 1.02230608e+00, 1.02494556e+00,
        1.02803178e+00]),
 array([8.41922891e-09, 1.03962180e-03, 1.02030288e+00, 1.02323753e+00,
        1.02635391e+00]),
 array([2.64801887e-06, 2.40865578e-03, 1.02320048e+00, 1.02601125e+00,
        1.02917453e+00]))

In [29]:
print_mode_and_hdi(f_tau10, display_cols = ['\mu','\delta','w_{2n+1}','w_{2n+1^*}', 'w_{2n^*}'],hdi_p=.3)

$\mu=1.674\times10^{-6} (2.501\times10^{-7}-1.741\times10^{-6})$,
$\delta=1.482\times10^{-3} (1.281\times10^{-3}-2.074\times10^{-3})$,
$w_{2n+1}=1.022 (1.021-1.023)$,
$w_{2n+1^*}=1.025 (1.024-1.025)$,
$w_{2n^*}=1.028 (1.027-1.029)$,


In [30]:
print_mode_and_hdi(f_tau100, display_cols = ['\mu','\delta','w_{2n+1}','w_{2n+1^*}', 'w_{2n^*}'],hdi_p=.3)

$\mu=3.81\times10^{-7} (1.508\times10^{-7}-4.995\times10^{-7})$,
$\delta=7.824\times10^{-4} (6.616\times10^{-4}-1.462\times10^{-3})$,
$w_{2n+1}=1.022 (1.021-1.023)$,
$w_{2n+1^*}=1.024 (1.024-1.025)$,
$w_{2n^*}=1.028 (1.027-1.029)$,
